# Tutorial for MOFF single HP1alpha dimer simulation.

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit

sys.path.append('../../')
from openabc.forcefields.parsers import MOFFParser
from openabc.forcefields import MOFFMRGModel

# set simulation platform
platform_name = 'CPU'

We simulate a single HP1alpha dimer with MOFF force field. We start from the atomistic model (hp1a.pdb) to build the CA model and apply MOFF force field. For your reference, we also provide hp1a.itp file, which is the GROMACS topology file. 

In [2]:
hp1alpha_dimer_parser = MOFFParser.from_atomistic_pdb('input-pdb/hp1a.pdb', 'hp1alpha_dimer_CA.pdb')

Parse molecule with default settings.
Get native pairs with shadow algorithm.


As shadow algorithm will find redundnat native pairs within disordered domains, we need to remove those redundant pairs. We only keep native pairs within CD, CSD, or between two CSDs. The CD domain is 17-72, and CSD domain is 115-176 (here indices start from 1), and there are 191 residues in each chain.

In [3]:
old_native_pairs = hp1alpha_dimer_parser.native_pairs.copy()
new_native_pairs = pd.DataFrame(columns=old_native_pairs.columns)
cd1 = np.arange(16, 72)
csd1 = np.arange(114, 176)
n_atoms_per_hp1alpha_dimer = len(hp1alpha_dimer_parser.atoms.index)
print(f'There are {n_atoms_per_hp1alpha_dimer} CA atoms in each HP1alpha dimer.')
cd2 = cd1 + int(n_atoms_per_hp1alpha_dimer/2)
csd2 = csd1 + int(n_atoms_per_hp1alpha_dimer/2)
for i, row in old_native_pairs.iterrows():
    a1, a2 = int(row['a1']), int(row['a2'])
    if a1 > a2:
        a1, a2 = a2, a1
    flag1 = ((a1 in cd1) and (a2 in cd1)) or ((a1 in csd1) and (a2 in csd1))
    flag2 = ((a1 in cd2) and (a2 in cd2)) or ((a1 in csd2) and (a2 in csd2))
    flag3 = ((a1 in csd1) and (a2 in csd2))
    if flag1 or flag2 or flag3:
        new_native_pairs.loc[len(new_native_pairs.index)] = row
hp1alpha_dimer_parser.native_pairs = new_native_pairs
hp1alpha_dimer_parser.parse_exclusions() # update exclusions based on the new native pairs

There are 382 CA atoms in each HP1alpha dimer.


We can read the bonded interactin parameters, which are pandas dataframes set as attributes of `hp1alpha_dimer_parser`. 

We print the first 5 rows of protein bonds, angles, dihedrals, and native pairs to take a look.

In [4]:
print(hp1alpha_dimer_parser.protein_bonds.head())
print(hp1alpha_dimer_parser.protein_angles.head())
print(hp1alpha_dimer_parser.protein_dihedrals.head())
print(hp1alpha_dimer_parser.native_pairs.head())

   a1  a2    r0  k_bond
0   0   1  0.38    1000
1   1   2  0.38    1000
2   2   3  0.38    1000
3   3   4  0.38    1000
4   4   5  0.38    1000
   a1  a2  a3    theta0  k_angle
0   0   1   2  1.994760      120
1   1   2   3  2.271120      120
2   2   3   4  2.179586      120
3   3   4   5  1.924860      120
4   4   5   6  2.371727      120
    a1   a2   a3   a4  periodicity      phi0  k_dihedral
0  0.0  1.0  2.0  3.0          1.0  0.653916         3.0
1  0.0  1.0  2.0  3.0          3.0  1.961747         1.5
2  1.0  2.0  3.0  4.0          1.0  0.320945         3.0
3  1.0  2.0  3.0  4.0          3.0  0.962834         1.5
4  2.0  3.0  4.0  5.0          1.0  0.887062         3.0
     a1    a2        mu  epsilon
0  18.0  41.0  0.812619      3.0
1  18.0  42.0  0.751187      3.0
2  19.0  40.0  0.576995      3.0
3  19.0  41.0  0.518028      3.0
4  19.0  42.0  0.549097      3.0


Now we can do the simulation. We append `hp1alpha_dimer_parser` to `protein`, and object `protein` includes `protein_bonds`, `protein_angles`, `protein_dihedrals`, and `native_pairs` as attributes. Note we use stronger native pairs (epsilon = 6.0 instead of 3.0). We directly change the epsilon parameter in `protein.native_pairs`. 

In [5]:
protein = MOFFMRGModel()
protein.append_mol(hp1alpha_dimer_parser)
protein.native_pairs.loc[:, 'epsilon'] = 6.0
top = app.PDBFile('hp1alpha_dimer_CA.pdb').getTopology()
protein.create_system(top)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
protein.add_protein_bonds(force_group=1)
protein.add_protein_angles(force_group=2)
protein.add_protein_dihedrals(force_group=3)
protein.add_native_pairs(force_group=4)
protein.add_contacts(force_group=5)
protein.add_elec_switch(salt_conc, temperature, force_group=6)
protein.save_system('hp1alpha_dimer_CA_system.xml')
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_dimer.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Add protein angles.
Add protein dihedrals.
Add native pairs.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,-2429.7341892797726,1141.8392285198042,-1287.8949607599684,240.30042536176086,0
200,2.0000000000000013,-2157.462022865531,1318.8005412701336,-838.6614815953972,277.54198937914407,97.3
300,2.99999999999998,-2004.473335575141,1373.514822785564,-630.9585127895768,289.05662716100153,96.8
400,3.9999999999999587,-2055.2423740849786,1416.0940706740448,-639.1483034109337,298.01744329310225,96.8
500,4.999999999999938,-1970.3332647662432,1359.7083877499024,-610.6248770163409,286.15105855823083,96.7


One useful setting is to apply rigid bodies. We use HP1alpha dimer as an example to show how to manipulate rigid body with our tool. The rigid body code is written by Peter Eastman, and we provide useful methods to help remove bonded interactions within the rigid body and make the simulation more efficient. The rigid body is realized by representing each rigid body with few real sites and certain restraints, while viewing the rest atoms as virtual sites.

In the following sessions, we rigidize each CD domain and two CSDs together (i.e. 3 rigid bodies in all) based on the native configuration. All the bonded interactions (bonds, angles, and dihedrals) within each rigid body will be removed to save computational resources. Additionally we do not need to add native pairs as they are all within the rigid bodies. 

In [6]:
protein = MOFFMRGModel()
protein.append_mol(hp1alpha_dimer_parser)
top = app.PDBFile('hp1alpha_dimer_CA.pdb').getTopology()
protein.create_system(top)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
rigid_coord = init_coord # set rigid body coordinates
rigid_bodies = [cd1.tolist(), cd2.tolist(), csd1.tolist() + csd2.tolist()] # 3 rigid bodies
protein.set_rigid_bodies(rigid_coord, rigid_bodies)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
protein.add_protein_bonds(force_group=1)
protein.add_protein_angles(force_group=2)
protein.add_protein_dihedrals(force_group=3)
protein.add_contacts(force_group=4)
protein.add_elec_switch(salt_conc, temperature, force_group=5)
protein.save_system('rigid_hp1alpha_dimer_CA_system.xml')
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('hp1alpha_dimer_CA.pdb').getPositions()
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_dimer.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Add protein angles.
Add protein dihedrals.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,157244.6628354427,473.51625512495525,157718.17909056766,251.4389190061399,0
200,2.0000000000000013,157292.44174817062,551.5841225495357,157844.02587072016,292.8932513165929,94.8
300,2.99999999999998,157370.94668979783,555.035950335312,157925.98264013315,294.72618490156964,90.5
400,3.9999999999999587,157464.28232542248,525.3285004925244,157989.610825915,278.9514168887409,89.5
500,4.999999999999938,157399.93367878505,523.522375608721,157923.45605439376,277.9923577572781,90.7
